# MoveNet Lightningによる姿勢推定 - Google Colab版

このノートブックでは、TensorFlow HubのMoveNet Lightningモデルを使用してリアルタイム姿勢推定を行います。MoveNet Lightningは軽量で高速な姿勢推定モデルです。

## 特徴
- 高速なリアルタイム姿勢推定
- 17個のキーポイントの検出
- 軽量モデル（Lightning版）
- 画像・動画両方に対応
- 信頼度スコア付きの結果

## 必要な環境
- Google Colab（推奨）
- Python 3.8以上
- TensorFlow 2.x

## 1. 必要なライブラリのインストール

MoveNet Lightningを使用するために必要なライブラリをインストールします。

In [ ]:
# TensorFlowとTensorFlow Hubをインストール
!pip install tensorflow tensorflow-hub

# その他の必要なライブラリをインストール
!pip install opencv-python-headless pillow matplotlib numpy

# インストール完了の確認
print("✅ 全てのライブラリのインストールが完了しました！")

## 2. ライブラリのインポートと設定

姿勢推定に必要なライブラリをインポートします。

In [ ]:
# 必要なライブラリをインポート
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import os
from google.colab import files
import io

# GPU利用可能性の確認
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")
print("✅ 全てのライブラリのインポートが完了しました！")

## 3. MoveNet Lightningモデルの読み込み

TensorFlow HubからMoveNet Lightningモデルを読み込みます。

In [ ]:
# MoveNet Lightning モデルをTensorFlow Hubから読み込み
model_url = "https://tfhub.dev/google/movenet/singlepose/lightning/4"
print("🔄 MoveNet Lightningモデルを読み込み中...")

# モデルをロード
movenet = hub.load(model_url)
movenet_fn = movenet.signatures['serving_default']

print("✅ MoveNet Lightningモデルの読み込みが完了しました！")
print("📊 モデル情報:")
print("   - 入力サイズ: 192x192")
print("   - 検出キーポイント数: 17個")
print("   - 出力形式: [y, x, confidence] × 17")

## 4. キーポイント定義と描画関数

MoveNetが検出する17個のキーポイントの定義と、結果を可視化するための関数を定義します。

In [ ]:
# MoveNetのキーポイント定義（17個）
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

# スケルトンの接続関係
CONNECTIONS = [
    # 顔
    (0, 1), (0, 2), (1, 3), (2, 4),
    # 胴体
    (5, 6), (5, 11), (6, 12), (11, 12),
    # 左腕
    (5, 7), (7, 9),
    # 右腕
    (6, 8), (8, 10),
    # 左脚
    (11, 13), (13, 15),
    # 右脚
    (12, 14), (14, 16)
]

def draw_keypoints_and_skeleton(image, keypoints, confidence_threshold=0.3):
    """
    画像にキーポイントとスケルトンを描画する関数
    """
    image_copy = image.copy()
    height, width = image_copy.shape[:2]
    
    # キーポイントを描画
    for i, (y, x, conf) in enumerate(keypoints):
        if conf > confidence_threshold:
            # 座標を画像サイズにスケール
            x_coord = int(x * width)
            y_coord = int(y * height)
            
            # キーポイントを円で描画
            cv2.circle(image_copy, (x_coord, y_coord), 5, (0, 255, 0), -1)
            cv2.circle(image_copy, (x_coord, y_coord), 7, (0, 0, 255), 2)
    
    # スケルトンを描画
    for connection in CONNECTIONS:
        point1_idx, point2_idx = connection
        
        y1, x1, conf1 = keypoints[point1_idx]
        y2, x2, conf2 = keypoints[point2_idx]
        
        if conf1 > confidence_threshold and conf2 > confidence_threshold:
            x1_coord = int(x1 * width)
            y1_coord = int(y1 * height)
            x2_coord = int(x2 * width)
            y2_coord = int(y2 * height)
            
            cv2.line(image_copy, (x1_coord, y1_coord), (x2_coord, y2_coord), (255, 0, 0), 2)
    
    return image_copy

print("✅ キーポイント定義と描画関数の準備が完了しました！")

## 5. 姿勢推定実行関数

画像に対してMoveNet Lightningで姿勢推定を実行する関数を定義します。

In [ ]:
def run_pose_estimation(image):
    """
    単一画像に対して姿勢推定を実行する関数
    """
    # 画像を192x192にリサイズ（MoveNet Lightningの入力サイズ）
    input_image = tf.expand_dims(image, axis=0)
    input_image = tf.cast(tf.image.resize_with_pad(input_image, 192, 192), dtype=tf.int32)
    
    # 姿勢推定を実行
    outputs = movenet_fn(input_image)
    keypoints = outputs['output_0'].numpy()
    
    # 結果を返す (17個のキーポイント × [y, x, confidence])
    return keypoints[0, 0, :, :]

def analyze_pose_results(keypoints, confidence_threshold=0.3):
    """
    姿勢推定結果を分析し、検出されたキーポイントの情報を表示
    """
    detected_points = []
    
    for name, idx in KEYPOINT_DICT.items():
        y, x, conf = keypoints[idx]
        if conf > confidence_threshold:
            detected_points.append((name, conf))
    
    print(f"🎯 検出されたキーポイント数: {len(detected_points)}/17")
    print("📊 検出されたキーポイント:")
    
    for name, conf in sorted(detected_points, key=lambda x: x[1], reverse=True):
        print(f"   - {name}: {conf:.3f}")
    
    return detected_points

print("✅ 姿勢推定実行関数の準備が完了しました！")

## 6. テスト画像のアップロードと準備

Google Colabに画像をアップロードして姿勢推定を実行します。

In [ ]:
# 画像ファイルをアップロード
print("📁 姿勢推定を行いたい画像ファイルを選択してアップロードしてください...")
print("💡 人物が写っている画像を選択することをお勧めします")
uploaded = files.upload()

# アップロードされた画像のパスを取得
uploaded_image_paths = list(uploaded.keys())
print(f"✅ {len(uploaded_image_paths)}個の画像がアップロードされました")

# アップロードされた画像を表示
if uploaded_image_paths:
    fig, axes = plt.subplots(1, min(3, len(uploaded_image_paths)), figsize=(15, 5))
    if len(uploaded_image_paths) == 1:
        axes = [axes]
    
    for i, image_path in enumerate(uploaded_image_paths[:3]):
        img = Image.open(image_path)
        if len(uploaded_image_paths) > 1:
            axes[i].imshow(img)
            axes[i].set_title(f'アップロード画像 {i+1}: {image_path}')
            axes[i].axis('off')
        else:
            axes[0].imshow(img)
            axes[0].set_title(f'アップロード画像: {image_path}')
            axes[0].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ 画像がアップロードされませんでした")

## 7. 姿勢推定の実行

アップロードした画像に対してMoveNet Lightningによる姿勢推定を実行します。

In [ ]:
# 姿勢推定を実行
if uploaded_image_paths:
    results_list = []
    
    for image_path in uploaded_image_paths:
        print(f"🔍 {image_path} の姿勢推定を実行中...")
        
        # 画像を読み込み
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 姿勢推定を実行
        keypoints = run_pose_estimation(image_rgb)
        results_list.append((image_rgb, keypoints))
        
        print(f"✅ 姿勢推定完了: {image_path}")
        
        # 結果を分析
        detected_points = analyze_pose_results(keypoints)
        print()
    
    print("🎉 全ての画像の姿勢推定が完了しました！")
else:
    print("❌ 姿勢推定する画像がありません。画像をアップロードしてください。")

## 8. 姿勢推定結果の可視化

検出されたキーポイントとスケルトンを元の画像に重ねて表示します。

In [ ]:
# 姿勢推定結果を可視化
if uploaded_image_paths and results_list:
    
    # 画像数に応じてsubplotを調整
    num_images = len(uploaded_image_paths)
    cols = min(2, num_images)
    rows = (num_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows * 2, cols, figsize=(15, 8 * rows))
    if num_images == 1:
        axes = axes.reshape(-1, 1)
    elif rows == 1:
        axes = axes.reshape(2, -1)
    
    for i, (image_path, (image_rgb, keypoints)) in enumerate(zip(uploaded_image_paths, results_list)):
        col = i % cols
        
        # 元画像を表示
        axes[0, col].imshow(image_rgb)
        axes[0, col].set_title(f'元画像: {image_path}')
        axes[0, col].axis('off')
        
        # 姿勢推定結果を重ねた画像を表示
        annotated_image = draw_keypoints_and_skeleton(image_rgb, keypoints)
        axes[1, col].imshow(annotated_image)
        axes[1, col].set_title(f'姿勢推定結果: {image_path}')
        axes[1, col].axis('off')
    
    # 余ったsubplotがあれば非表示にする
    for i in range(num_images, cols):
        if i < cols:
            axes[0, i].axis('off')
            axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("🎨 姿勢推定結果の可視化が完了しました！")
    print("📊 表示されている要素の説明:")
    print("   - 緑色の円: 検出されたキーポイント")
    print("   - 赤色の円: キーポイントの境界")
    print("   - 青色の線: スケルトン（骨格）")
    print("   - 信頼度が低いキーポイントは表示されません")
else:
    print("❌ 表示する結果がありません")

## 9. 詳細分析（オプション）

検出されたキーポイントの詳細情報を確認します。

In [ ]:
# 詳細分析
if uploaded_image_paths and results_list:
    print("📋 詳細分析結果")
    print("=" * 50)
    
    for i, (image_path, (image_rgb, keypoints)) in enumerate(zip(uploaded_image_paths, results_list)):
        print(f"\n🖼️  画像 {i+1}: {image_path}")
        print("-" * 30)
        
        # 各キーポイントの詳細情報
        print("📍 全キーポイントの詳細:")
        for name, idx in KEYPOINT_DICT.items():
            y, x, conf = keypoints[idx]
            status = "✅ 検出" if conf > 0.3 else "❌ 未検出"
            print(f"   {name:15}: {status} (信頼度: {conf:.3f}, 座標: ({x:.3f}, {y:.3f}))")
        
        # 検出率の計算
        detected_count = sum(1 for _, _, conf in keypoints if conf > 0.3)
        detection_rate = detected_count / 17 * 100
        print(f"\n📊 検出率: {detection_rate:.1f}% ({detected_count}/17 キーポイント)")
        
        # 平均信頼度
        avg_confidence = np.mean([conf for _, _, conf in keypoints if conf > 0.3])
        print(f"📈 平均信頼度: {avg_confidence:.3f}")
        
        print()
    
    print("✅ 詳細分析が完了しました！")
else:
    print("❌ 分析する結果がありません")

## 🎯 使用方法のまとめ

1. **セルを順番に実行**: 上から順番にセルを実行してください
2. **画像をアップロード**: セル6で人物が写った画像をアップロードしてください
3. **結果を確認**: セル8で姿勢推定結果を可視化して確認できます
4. **詳細分析**: セル9で各キーポイントの詳細情報を確認できます

## 📝 注意事項

- **最適な画像**: 人物が明確に写っている画像を使用することをお勧めします
- **信頼度閾値**: デフォルトは0.3ですが、必要に応じて調整できます
- **モデルの制限**: 単一人物の姿勢推定にのみ対応しています
- **GPU利用**: Google ColabでGPUランタイムを使用するとより高速に動作します

## 🔗 参考リンク

- [MoveNet: Ultra fast and accurate pose detection model](https://blog.tensorflow.org/2021/05/next-generation-pose-detection-with-movenet-and-tensorflowjs.html)
- [TensorFlow Hub - MoveNet](https://tfhub.dev/google/movenet/singlepose/lightning/4)
- [TensorFlow Pose Estimation Guide](https://www.tensorflow.org/lite/examples/pose_estimation/overview)

# MoveNet Lightningによる姿勢推定 - Google Colab版

このノートブックでは、TensorFlow HubのMoveNet Lightningモデルを使用してリアルタイム姿勢推定を行います。MoveNet Lightningは軽量で高速な姿勢推定モデルです。

## 特徴
- 高速なリアルタイム姿勢推定
- 17個のキーポイントの検出
- 軽量モデル（Lightning版）
- 画像・動画両方に対応
- 信頼度スコア付きの結果

## 必要な環境
- Google Colab（推奨）
- Python 3.8以上
- TensorFlow 2.x

## 1. 必要なライブラリのインストール

MoveNet Lightningを使用するために必要なライブラリをインストールします。

In [ ]:
# TensorFlowとTensorFlow Hubをインストール
!pip install tensorflow tensorflow-hub

# その他の必要なライブラリをインストール
!pip install opencv-python-headless pillow matplotlib numpy

# インストール完了の確認
print("✅ 全てのライブラリのインストールが完了しました！")

## 2. ライブラリのインポートと設定

姿勢推定に必要なライブラリをインポートします。

In [ ]:
# 必要なライブラリをインポート
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import os
from google.colab import files
import io

# GPU利用可能性の確認
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")
print("✅ 全てのライブラリのインポートが完了しました！")

## 3. MoveNet Lightningモデルの読み込み

TensorFlow HubからMoveNet Lightningモデルを読み込みます。

In [ ]:
# MoveNet Lightning モデルをTensorFlow Hubから読み込み
model_url = "https://tfhub.dev/google/movenet/singlepose/lightning/4"
print("🔄 MoveNet Lightningモデルを読み込み中...")

# モデルをロード
movenet = hub.load(model_url)
movenet_fn = movenet.signatures['serving_default']

print("✅ MoveNet Lightningモデルの読み込みが完了しました！")
print("📊 モデル情報:")
print("   - 入力サイズ: 192x192")
print("   - 検出キーポイント数: 17個")
print("   - 出力形式: [y, x, confidence] × 17")

## 4. キーポイント定義と描画関数

MoveNetが検出する17個のキーポイントの定義と、結果を可視化するための関数を定義します。

In [ ]:
# MoveNetのキーポイント定義（17個）
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

# スケルトンの接続関係
CONNECTIONS = [
    # 顔
    (0, 1), (0, 2), (1, 3), (2, 4),
    # 胴体
    (5, 6), (5, 11), (6, 12), (11, 12),
    # 左腕
    (5, 7), (7, 9),
    # 右腕
    (6, 8), (8, 10),
    # 左脚
    (11, 13), (13, 15),
    # 右脚
    (12, 14), (14, 16)
]

def draw_keypoints_and_skeleton(image, keypoints, confidence_threshold=0.3):
    """
    画像にキーポイントとスケルトンを描画する関数
    """
    image_copy = image.copy()
    height, width = image_copy.shape[:2]
    
    # キーポイントを描画
    for i, (y, x, conf) in enumerate(keypoints):
        if conf > confidence_threshold:
            # 座標を画像サイズにスケール
            x_coord = int(x * width)
            y_coord = int(y * height)
            
            # キーポイントを円で描画
            cv2.circle(image_copy, (x_coord, y_coord), 5, (0, 255, 0), -1)
            cv2.circle(image_copy, (x_coord, y_coord), 7, (0, 0, 255), 2)
    
    # スケルトンを描画
    for connection in CONNECTIONS:
        point1_idx, point2_idx = connection
        
        y1, x1, conf1 = keypoints[point1_idx]
        y2, x2, conf2 = keypoints[point2_idx]
        
        if conf1 > confidence_threshold and conf2 > confidence_threshold:
            x1_coord = int(x1 * width)
            y1_coord = int(y1 * height)
            x2_coord = int(x2 * width)
            y2_coord = int(y2 * height)
            
            cv2.line(image_copy, (x1_coord, y1_coord), (x2_coord, y2_coord), (255, 0, 0), 2)
    
    return image_copy

print("✅ キーポイント定義と描画関数の準備が完了しました！")

## 5. 姿勢推定実行関数

画像に対してMoveNet Lightningで姿勢推定を実行する関数を定義します。

In [ ]:
def run_pose_estimation(image):
    """
    単一画像に対して姿勢推定を実行する関数
    """
    # 画像を192x192にリサイズ（MoveNet Lightningの入力サイズ）
    input_image = tf.expand_dims(image, axis=0)
    input_image = tf.cast(tf.image.resize_with_pad(input_image, 192, 192), dtype=tf.int32)
    
    # 姿勢推定を実行
    outputs = movenet_fn(input_image)
    keypoints = outputs['output_0'].numpy()
    
    # 結果を返す (17個のキーポイント × [y, x, confidence])
    return keypoints[0, 0, :, :]

def analyze_pose_results(keypoints, confidence_threshold=0.3):
    """
    姿勢推定結果を分析し、検出されたキーポイントの情報を表示
    """
    detected_points = []
    
    for name, idx in KEYPOINT_DICT.items():
        y, x, conf = keypoints[idx]
        if conf > confidence_threshold:
            detected_points.append((name, conf))
    
    print(f"🎯 検出されたキーポイント数: {len(detected_points)}/17")
    print("📊 検出されたキーポイント:")
    
    for name, conf in sorted(detected_points, key=lambda x: x[1], reverse=True):
        print(f"   - {name}: {conf:.3f}")
    
    return detected_points

print("✅ 姿勢推定実行関数の準備が完了しました！")

## 6. テスト画像のアップロードと準備

Google Colabに画像をアップロードして姿勢推定を実行します。

In [ ]:
# 画像ファイルをアップロード
print("📁 姿勢推定を行いたい画像ファイルを選択してアップロードしてください...")
print("💡 人物が写っている画像を選択することをお勧めします")
uploaded = files.upload()

# アップロードされた画像のパスを取得
uploaded_image_paths = list(uploaded.keys())
print(f"✅ {len(uploaded_image_paths)}個の画像がアップロードされました")

# アップロードされた画像を表示
if uploaded_image_paths:
    fig, axes = plt.subplots(1, min(3, len(uploaded_image_paths)), figsize=(15, 5))
    if len(uploaded_image_paths) == 1:
        axes = [axes]
    
    for i, image_path in enumerate(uploaded_image_paths[:3]):
        img = Image.open(image_path)
        if len(uploaded_image_paths) > 1:
            axes[i].imshow(img)
            axes[i].set_title(f'アップロード画像 {i+1}: {image_path}')
            axes[i].axis('off')
        else:
            axes[0].imshow(img)
            axes[0].set_title(f'アップロード画像: {image_path}')
            axes[0].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ 画像がアップロードされませんでした")

## 7. 姿勢推定の実行

アップロードした画像に対してMoveNet Lightningによる姿勢推定を実行します。

In [ ]:
# 姿勢推定を実行
if uploaded_image_paths:
    results_list = []
    
    for image_path in uploaded_image_paths:
        print(f"🔍 {image_path} の姿勢推定を実行中...")
        
        # 画像を読み込み
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 姿勢推定を実行
        keypoints = run_pose_estimation(image_rgb)
        results_list.append((image_rgb, keypoints))
        
        print(f"✅ 姿勢推定完了: {image_path}")
        
        # 結果を分析
        detected_points = analyze_pose_results(keypoints)
        print()
    
    print("🎉 全ての画像の姿勢推定が完了しました！")
else:
    print("❌ 姿勢推定する画像がありません。画像をアップロードしてください。")

## 8. 姿勢推定結果の可視化

検出されたキーポイントとスケルトンを元の画像に重ねて表示します。

In [ ]:
# 姿勢推定結果を可視化
if uploaded_image_paths and results_list:
    
    # 画像数に応じてsubplotを調整
    num_images = len(uploaded_image_paths)
    cols = min(2, num_images)
    rows = (num_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows * 2, cols, figsize=(15, 8 * rows))
    if num_images == 1:
        axes = axes.reshape(-1, 1)
    elif rows == 1:
        axes = axes.reshape(2, -1)
    
    for i, (image_path, (image_rgb, keypoints)) in enumerate(zip(uploaded_image_paths, results_list)):
        col = i % cols
        
        # 元画像を表示
        axes[0, col].imshow(image_rgb)
        axes[0, col].set_title(f'元画像: {image_path}')
        axes[0, col].axis('off')
        
        # 姿勢推定結果を重ねた画像を表示
        annotated_image = draw_keypoints_and_skeleton(image_rgb, keypoints)
        axes[1, col].imshow(annotated_image)
        axes[1, col].set_title(f'姿勢推定結果: {image_path}')
        axes[1, col].axis('off')
    
    # 余ったsubplotがあれば非表示にする
    for i in range(num_images, cols):
        if i < cols:
            axes[0, i].axis('off')
            axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("🎨 姿勢推定結果の可視化が完了しました！")
    print("📊 表示されている要素の説明:")
    print("   - 緑色の円: 検出されたキーポイント")
    print("   - 赤色の円: キーポイントの境界")
    print("   - 青色の線: スケルトン（骨格）")
    print("   - 信頼度が低いキーポイントは表示されません")
else:
    print("❌ 表示する結果がありません")

## 9. 詳細分析（オプション）

検出されたキーポイントの詳細情報を確認します。

In [ ]:
# 詳細分析
if uploaded_image_paths and results_list:
    print("📋 詳細分析結果")
    print("=" * 50)
    
    for i, (image_path, (image_rgb, keypoints)) in enumerate(zip(uploaded_image_paths, results_list)):
        print(f"\n🖼️  画像 {i+1}: {image_path}")
        print("-" * 30)
        
        # 各キーポイントの詳細情報
        print("📍 全キーポイントの詳細:")
        for name, idx in KEYPOINT_DICT.items():
            y, x, conf = keypoints[idx]
            status = "✅ 検出" if conf > 0.3 else "❌ 未検出"
            print(f"   {name:15}: {status} (信頼度: {conf:.3f}, 座標: ({x:.3f}, {y:.3f}))")
        
        # 検出率の計算
        detected_count = sum(1 for _, _, conf in keypoints if conf > 0.3)
        detection_rate = detected_count / 17 * 100
        print(f"\n📊 検出率: {detection_rate:.1f}% ({detected_count}/17 キーポイント)")
        
        # 平均信頼度
        avg_confidence = np.mean([conf for _, _, conf in keypoints if conf > 0.3])
        print(f"📈 平均信頼度: {avg_confidence:.3f}")
        
        print()
    
    print("✅ 詳細分析が完了しました！")
else:
    print("❌ 分析する結果がありません")

## 🎯 使用方法のまとめ

1. **順番に実行**: 上から順番にセルを実行してください
2. **画像をアップロード**: 6番目のセルで人物が写った画像をアップロードしてください
3. **結果を確認**: 8番目のセルで姿勢推定結果を可視化して確認できます
4. **詳細分析**: 9番目のセルで各キーポイントの詳細情報を確認できます

## 📝 注意事項

- **最適な画像**: 人物が明確に写っている画像を使用することをお勧めします
- **信頼度閾値**: デフォルトは0.3ですが、必要に応じて調整できます
- **モデルの制限**: 単一人物の姿勢推定にのみ対応しています
- **GPU利用**: Google ColabでGPUランタイムを使用するとより高速に動作します

## 🔗 参考リンク

- [MoveNet: Ultra fast and accurate pose detection model](https://blog.tensorflow.org/2021/05/next-generation-pose-detection-with-movenet-and-tensorflowjs.html)
- [TensorFlow Hub - MoveNet](https://tfhub.dev/google/movenet/singlepose/lightning/4)
- [TensorFlow Pose Estimation Guide](https://www.tensorflow.org/lite/examples/pose_estimation/overview)

# MoveNet Lightningによる姿勢推定 - Google Colab版

このノートブックでは、TensorFlow HubのMoveNet Lightningモデルを使用してリアルタイム姿勢推定を行います。MoveNet Lightningは軽量で高速な姿勢推定モデルです。

## 特徴
- 高速なリアルタイム姿勢推定
- 17個のキーポイントの検出
- 軽量モデル（Lightning版）
- 画像・動画両方に対応
- 信頼度スコア付きの結果

## 必要な環境
- Google Colab（推奨）
- Python 3.8以上
- TensorFlow 2.x

## 1. 必要なライブラリのインストール

MoveNet Lightningを使用するために必要なライブラリをインストールします。

In [ ]:
# TensorFlowとTensorFlow Hubをインストール
!pip install tensorflow tensorflow-hub

# その他の必要なライブラリをインストール
!pip install opencv-python-headless pillow matplotlib numpy

# インストール完了の確認
print("✅ 全てのライブラリのインストールが完了しました！")

## 2. ライブラリのインポートと設定

姿勢推定に必要なライブラリをインポートします。

In [ ]:
# 必要なライブラリをインポート
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import os
from google.colab import files
import io

# GPU利用可能性の確認
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")
print("✅ 全てのライブラリのインポートが完了しました！")

## 3. MoveNet Lightningモデルの読み込み

TensorFlow HubからMoveNet Lightningモデルを読み込みます。

In [ ]:
# MoveNet Lightning モデルをTensorFlow Hubから読み込み
model_url = "https://tfhub.dev/google/movenet/singlepose/lightning/4"
print("🔄 MoveNet Lightningモデルを読み込み中...")

# モデルをロード
movenet = hub.load(model_url)
movenet_fn = movenet.signatures['serving_default']

print("✅ MoveNet Lightningモデルの読み込みが完了しました！")
print("📊 モデル情報:")
print("   - 入力サイズ: 192x192")
print("   - 検出キーポイント数: 17個")
print("   - 出力形式: [y, x, confidence] × 17")

## 4. キーポイント定義と描画関数

MoveNetが検出する17個のキーポイントの定義と、結果を可視化するための関数を定義します。

In [ ]:
# MoveNetのキーポイント定義（17個）
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

# スケルトンの接続関係
CONNECTIONS = [
    # 顔
    (0, 1), (0, 2), (1, 3), (2, 4),
    # 胴体
    (5, 6), (5, 11), (6, 12), (11, 12),
    # 左腕
    (5, 7), (7, 9),
    # 右腕
    (6, 8), (8, 10),
    # 左脚
    (11, 13), (13, 15),
    # 右脚
    (12, 14), (14, 16)
]

def draw_keypoints_and_skeleton(image, keypoints, confidence_threshold=0.3):
    """
    画像にキーポイントとスケルトンを描画する関数
    """
    image_copy = image.copy()
    height, width = image_copy.shape[:2]
    
    # キーポイントを描画
    for i, (y, x, conf) in enumerate(keypoints):
        if conf > confidence_threshold:
            # 座標を画像サイズにスケール
            x_coord = int(x * width)
            y_coord = int(y * height)
            
            # キーポイントを円で描画
            cv2.circle(image_copy, (x_coord, y_coord), 5, (0, 255, 0), -1)
            cv2.circle(image_copy, (x_coord, y_coord), 7, (0, 0, 255), 2)
    
    # スケルトンを描画
    for connection in CONNECTIONS:
        point1_idx, point2_idx = connection
        
        y1, x1, conf1 = keypoints[point1_idx]
        y2, x2, conf2 = keypoints[point2_idx]
        
        if conf1 > confidence_threshold and conf2 > confidence_threshold:
            x1_coord = int(x1 * width)
            y1_coord = int(y1 * height)
            x2_coord = int(x2 * width)
            y2_coord = int(y2 * height)
            
            cv2.line(image_copy, (x1_coord, y1_coord), (x2_coord, y2_coord), (255, 0, 0), 2)
    
    return image_copy

print("✅ キーポイント定義と描画関数の準備が完了しました！")

## 5. 姿勢推定実行関数

画像に対してMoveNet Lightningで姿勢推定を実行する関数を定義します。

In [ ]:
def run_pose_estimation(image):
    """
    単一画像に対して姿勢推定を実行する関数
    """
    # 画像を192x192にリサイズ（MoveNet Lightningの入力サイズ）
    input_image = tf.expand_dims(image, axis=0)
    input_image = tf.cast(tf.image.resize_with_pad(input_image, 192, 192), dtype=tf.int32)
    
    # 姿勢推定を実行
    outputs = movenet_fn(input_image)
    keypoints = outputs['output_0'].numpy()
    
    # 結果を返す (17個のキーポイント × [y, x, confidence])
    return keypoints[0, 0, :, :]

def analyze_pose_results(keypoints, confidence_threshold=0.3):
    """
    姿勢推定結果を分析し、検出されたキーポイントの情報を表示
    """
    detected_points = []
    
    for name, idx in KEYPOINT_DICT.items():
        y, x, conf = keypoints[idx]
        if conf > confidence_threshold:
            detected_points.append((name, conf))
    
    print(f"🎯 検出されたキーポイント数: {len(detected_points)}/17")
    print("📊 検出されたキーポイント:")
    
    for name, conf in sorted(detected_points, key=lambda x: x[1], reverse=True):
        print(f"   - {name}: {conf:.3f}")
    
    return detected_points

print("✅ 姿勢推定実行関数の準備が完了しました！")

## 6. テスト画像のアップロードと準備

Google Colabに画像をアップロードして姿勢推定を実行します。

In [ ]:
# 画像ファイルをアップロード
print("📁 姿勢推定を行いたい画像ファイルを選択してアップロードしてください...")
print("💡 人物が写っている画像を選択することをお勧めします")
uploaded = files.upload()

# アップロードされた画像のパスを取得
uploaded_image_paths = list(uploaded.keys())
print(f"✅ {len(uploaded_image_paths)}個の画像がアップロードされました")

# アップロードされた画像を表示
if uploaded_image_paths:
    fig, axes = plt.subplots(1, min(3, len(uploaded_image_paths)), figsize=(15, 5))
    if len(uploaded_image_paths) == 1:
        axes = [axes]
    
    for i, image_path in enumerate(uploaded_image_paths[:3]):
        img = Image.open(image_path)
        if len(uploaded_image_paths) > 1:
            axes[i].imshow(img)
            axes[i].set_title(f'アップロード画像 {i+1}: {image_path}')
            axes[i].axis('off')
        else:
            axes[0].imshow(img)
            axes[0].set_title(f'アップロード画像: {image_path}')
            axes[0].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ 画像がアップロードされませんでした")

## 7. 姿勢推定の実行

アップロードした画像に対してMoveNet Lightningによる姿勢推定を実行します。

In [ ]:
# 姿勢推定を実行
if uploaded_image_paths:
    results_list = []
    
    for image_path in uploaded_image_paths:
        print(f"🔍 {image_path} の姿勢推定を実行中...")
        
        # 画像を読み込み
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 姿勢推定を実行
        keypoints = run_pose_estimation(image_rgb)
        results_list.append((image_rgb, keypoints))
        
        print(f"✅ 姿勢推定完了: {image_path}")
        
        # 結果を分析
        detected_points = analyze_pose_results(keypoints)
        print()
    
    print("🎉 全ての画像の姿勢推定が完了しました！")
else:
    print("❌ 姿勢推定する画像がありません。画像をアップロードしてください。")

## 8. 姿勢推定結果の可視化

検出されたキーポイントとスケルトンを元の画像に重ねて表示します。

In [ ]:
# 姿勢推定結果を可視化
if uploaded_image_paths and results_list:
    
    # 画像数に応じてsubplotを調整
    num_images = len(uploaded_image_paths)
    cols = min(2, num_images)
    rows = (num_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows * 2, cols, figsize=(15, 8 * rows))
    if num_images == 1:
        axes = axes.reshape(-1, 1)
    elif rows == 1:
        axes = axes.reshape(2, -1)
    
    for i, (image_path, (image_rgb, keypoints)) in enumerate(zip(uploaded_image_paths, results_list)):
        col = i % cols
        
        # 元画像を表示
        axes[0, col].imshow(image_rgb)
        axes[0, col].set_title(f'元画像: {image_path}')
        axes[0, col].axis('off')
        
        # 姿勢推定結果を重ねた画像を表示
        annotated_image = draw_keypoints_and_skeleton(image_rgb, keypoints)
        axes[1, col].imshow(annotated_image)
        axes[1, col].set_title(f'姿勢推定結果: {image_path}')
        axes[1, col].axis('off')
    
    # 余ったsubplotがあれば非表示にする
    for i in range(num_images, cols):
        if i < cols:
            axes[0, i].axis('off')
            axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("🎨 姿勢推定結果の可視化が完了しました！")
    print("📊 表示されている要素の説明:")
    print("   - 緑色の円: 検出されたキーポイント")
    print("   - 赤色の円: キーポイントの境界")
    print("   - 青色の線: スケルトン（骨格）")
    print("   - 信頼度が低いキーポイントは表示されません")
else:
    print("❌ 表示する結果がありません")

## 9. 詳細分析（オプション）

検出されたキーポイントの詳細情報を確認します。

In [ ]:
# 詳細分析
if uploaded_image_paths and results_list:
    print("📋 詳細分析結果")
    print("=" * 50)
    
    for i, (image_path, (image_rgb, keypoints)) in enumerate(zip(uploaded_image_paths, results_list)):
        print(f"\n🖼️  画像 {i+1}: {image_path}")
        print("-" * 30)
        
        # 各キーポイントの詳細情報
        print("📍 全キーポイントの詳細:")
        for name, idx in KEYPOINT_DICT.items():
            y, x, conf = keypoints[idx]
            status = "✅ 検出" if conf > 0.3 else "❌ 未検出"
            print(f"   {name:15}: {status} (信頼度: {conf:.3f}, 座標: ({x:.3f}, {y:.3f}))")
        
        # 検出率の計算
        detected_count = sum(1 for _, _, conf in keypoints if conf > 0.3)
        detection_rate = detected_count / 17 * 100
        print(f"\n📊 検出率: {detection_rate:.1f}% ({detected_count}/17 キーポイント)")
        
        # 平均信頼度
        avg_confidence = np.mean([conf for _, _, conf in keypoints if conf > 0.3])
        print(f"📈 平均信頼度: {avg_confidence:.3f}")
        
        print()
    
    print("✅ 詳細分析が完了しました！")
else:
    print("❌ 分析する結果がありません")

## 🎯 使用方法のまとめ

1. **セルを順番に実行**: 上から順番にセルを実行してください
2. **画像をアップロード**: 6番目のセルで人物が写った画像をアップロードしてください
3. **結果を確認**: 8番目のセルで姿勢推定結果を可視化して確認できます
4. **詳細分析**: 9番目のセルで各キーポイントの詳細情報を確認できます

## 📝 注意事項

- **最適な画像**: 人物が明確に写っている画像を使用することをお勧めします
- **信頼度閾値**: デフォルトは0.3ですが、必要に応じて調整できます
- **モデルの制限**: 単一人物の姿勢推定にのみ対応しています
- **GPU利用**: Google ColabでGPUランタイムを使用するとより高速に動作します

## 🔗 参考リンク

- [MoveNet: Ultra fast and accurate pose detection model](https://blog.tensorflow.org/2021/05/next-generation-pose-detection-with-movenet-and-tensorflowjs.html)
- [TensorFlow Hub - MoveNet](https://tfhub.dev/google/movenet/singlepose/lightning/4)
- [TensorFlow Pose Estimation Guide](https://www.tensorflow.org/lite/examples/pose_estimation/overview)

# MoveNet Lightningによる姿勢推定 - Google Colab版

このノートブックでは、TensorFlow HubのMoveNet Lightningモデルを使用してリアルタイム姿勢推定を行います。MoveNet Lightningは軽量で高速な姿勢推定モデルです。

## 特徴
- 高速なリアルタイム姿勢推定
- 17個のキーポイントの検出
- 軽量モデル（Lightning版）
- 画像・動画両方に対応
- 信頼度スコア付きの結果

## 必要な環境
- Google Colab（推奨）
- Python 3.8以上
- TensorFlow 2.x

## 1. 必要なライブラリのインストール

MoveNet Lightningを使用するために必要なライブラリをインストールします。

In [ ]:
# TensorFlowとTensorFlow Hubをインストール
!pip install tensorflow tensorflow-hub

# その他の必要なライブラリをインストール
!pip install opencv-python-headless pillow matplotlib numpy

# インストール完了の確認
print("✅ 全てのライブラリのインストールが完了しました！")

## 2. ライブラリのインポートと設定

姿勢推定に必要なライブラリをインポートします。

In [ ]:
# 必要なライブラリをインポート
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import os
from google.colab import files
import io

# GPU利用可能性の確認
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")
print("✅ 全てのライブラリのインポートが完了しました！")

## 3. MoveNet Lightningモデルの読み込み

TensorFlow HubからMoveNet Lightningモデルを読み込みます。

In [ ]:
# MoveNet Lightning モデルをTensorFlow Hubから読み込み
model_url = "https://tfhub.dev/google/movenet/singlepose/lightning/4"
print("🔄 MoveNet Lightningモデルを読み込み中...")

# モデルをロード
movenet = hub.load(model_url)
movenet_fn = movenet.signatures['serving_default']

print("✅ MoveNet Lightningモデルの読み込みが完了しました！")
print("📊 モデル情報:")
print("   - 入力サイズ: 192x192")
print("   - 検出キーポイント数: 17個")
print("   - 出力形式: [y, x, confidence] × 17")

## 4. キーポイント定義と描画関数

MoveNetが検出する17個のキーポイントの定義と、結果を可視化するための関数を定義します。

In [ ]:
# MoveNetのキーポイント定義（17個）
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

# スケルトンの接続関係
CONNECTIONS = [
    # 顔
    (0, 1), (0, 2), (1, 3), (2, 4),
    # 胴体
    (5, 6), (5, 11), (6, 12), (11, 12),
    # 左腕
    (5, 7), (7, 9),
    # 右腕
    (6, 8), (8, 10),
    # 左脚
    (11, 13), (13, 15),
    # 右脚
    (12, 14), (14, 16)
]

def draw_keypoints_and_skeleton(image, keypoints, confidence_threshold=0.3):
    """
    画像にキーポイントとスケルトンを描画する関数
    """
    image_copy = image.copy()
    height, width = image_copy.shape[:2]
    
    # キーポイントを描画
    for i, (y, x, conf) in enumerate(keypoints):
        if conf > confidence_threshold:
            # 座標を画像サイズにスケール
            x_coord = int(x * width)
            y_coord = int(y * height)
            
            # キーポイントを円で描画
            cv2.circle(image_copy, (x_coord, y_coord), 5, (0, 255, 0), -1)
            cv2.circle(image_copy, (x_coord, y_coord), 7, (0, 0, 255), 2)
    
    # スケルトンを描画
    for connection in CONNECTIONS:
        point1_idx, point2_idx = connection
        
        y1, x1, conf1 = keypoints[point1_idx]
        y2, x2, conf2 = keypoints[point2_idx]
        
        if conf1 > confidence_threshold and conf2 > confidence_threshold:
            x1_coord = int(x1 * width)
            y1_coord = int(y1 * height)
            x2_coord = int(x2 * width)
            y2_coord = int(y2 * height)
            
            cv2.line(image_copy, (x1_coord, y1_coord), (x2_coord, y2_coord), (255, 0, 0), 2)
    
    return image_copy

print("✅ キーポイント定義と描画関数の準備が完了しました！")

## 5. 姿勢推定実行関数

画像に対してMoveNet Lightningで姿勢推定を実行する関数を定義します。

In [ ]:
def run_pose_estimation(image):
    """
    単一画像に対して姿勢推定を実行する関数
    """
    # 画像を192x192にリサイズ（MoveNet Lightningの入力サイズ）
    input_image = tf.expand_dims(image, axis=0)
    input_image = tf.cast(tf.image.resize_with_pad(input_image, 192, 192), dtype=tf.int32)
    
    # 姿勢推定を実行
    outputs = movenet_fn(input_image)
    keypoints = outputs['output_0'].numpy()
    
    # 結果を返す (17個のキーポイント × [y, x, confidence])
    return keypoints[0, 0, :, :]

def analyze_pose_results(keypoints, confidence_threshold=0.3):
    """
    姿勢推定結果を分析し、検出されたキーポイントの情報を表示
    """
    detected_points = []
    
    for name, idx in KEYPOINT_DICT.items():
        y, x, conf = keypoints[idx]
        if conf > confidence_threshold:
            detected_points.append((name, conf))
    
    print(f"🎯 検出されたキーポイント数: {len(detected_points)}/17")
    print("📊 検出されたキーポイント:")
    
    for name, conf in sorted(detected_points, key=lambda x: x[1], reverse=True):
        print(f"   - {name}: {conf:.3f}")
    
    return detected_points

print("✅ 姿勢推定実行関数の準備が完了しました！")

## 6. テスト画像のアップロードと準備

Google Colabに画像をアップロードして姿勢推定を実行します。

In [ ]:
# 画像ファイルをアップロード
print("📁 姿勢推定を行いたい画像ファイルを選択してアップロードしてください...")
print("💡 人物が写っている画像を選択することをお勧めします")
uploaded = files.upload()

# アップロードされた画像のパスを取得
uploaded_image_paths = list(uploaded.keys())
print(f"✅ {len(uploaded_image_paths)}個の画像がアップロードされました")

# アップロードされた画像を表示
if uploaded_image_paths:
    fig, axes = plt.subplots(1, min(3, len(uploaded_image_paths)), figsize=(15, 5))
    if len(uploaded_image_paths) == 1:
        axes = [axes]
    
    for i, image_path in enumerate(uploaded_image_paths[:3]):
        img = Image.open(image_path)
        if len(uploaded_image_paths) > 1:
            axes[i].imshow(img)
            axes[i].set_title(f'アップロード画像 {i+1}: {image_path}')
            axes[i].axis('off')
        else:
            axes[0].imshow(img)
            axes[0].set_title(f'アップロード画像: {image_path}')
            axes[0].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ 画像がアップロードされませんでした")

## 7. 姿勢推定の実行

アップロードした画像に対してMoveNet Lightningによる姿勢推定を実行します。

In [ ]:
# 姿勢推定を実行
if uploaded_image_paths:
    results_list = []
    
    for image_path in uploaded_image_paths:
        print(f"🔍 {image_path} の姿勢推定を実行中...")
        
        # 画像を読み込み
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 姿勢推定を実行
        keypoints = run_pose_estimation(image_rgb)
        results_list.append((image_rgb, keypoints))
        
        print(f"✅ 姿勢推定完了: {image_path}")
        
        # 結果を分析
        detected_points = analyze_pose_results(keypoints)
        print()
    
    print("🎉 全ての画像の姿勢推定が完了しました！")
else:
    print("❌ 姿勢推定する画像がありません。画像をアップロードしてください。")

## 8. 姿勢推定結果の可視化

検出されたキーポイントとスケルトンを元の画像に重ねて表示します。

In [ ]:
# 姿勢推定結果を可視化
if uploaded_image_paths and results_list:
    
    # 画像数に応じてsubplotを調整
    num_images = len(uploaded_image_paths)
    cols = min(2, num_images)
    rows = (num_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows * 2, cols, figsize=(15, 8 * rows))
    if num_images == 1:
        axes = axes.reshape(-1, 1)
    elif rows == 1:
        axes = axes.reshape(2, -1)
    
    for i, (image_path, (image_rgb, keypoints)) in enumerate(zip(uploaded_image_paths, results_list)):
        col = i % cols
        
        # 元画像を表示
        axes[0, col].imshow(image_rgb)
        axes[0, col].set_title(f'元画像: {image_path}')
        axes[0, col].axis('off')
        
        # 姿勢推定結果を重ねた画像を表示
        annotated_image = draw_keypoints_and_skeleton(image_rgb, keypoints)
        axes[1, col].imshow(annotated_image)
        axes[1, col].set_title(f'姿勢推定結果: {image_path}')
        axes[1, col].axis('off')
    
    # 余ったsubplotがあれば非表示にする
    for i in range(num_images, cols):
        if i < cols:
            axes[0, i].axis('off')
            axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("🎨 姿勢推定結果の可視化が完了しました！")
    print("📊 表示されている要素の説明:")
    print("   - 緑色の円: 検出されたキーポイント")
    print("   - 赤色の円: キーポイントの境界")
    print("   - 青色の線: スケルトン（骨格）")
    print("   - 信頼度が低いキーポイントは表示されません")
else:
    print("❌ 表示する結果がありません")

## 9. 詳細分析（オプション）

検出されたキーポイントの詳細情報を確認します。

In [ ]:
# 詳細分析
if uploaded_image_paths and results_list:
    print("📋 詳細分析結果")
    print("=" * 50)
    
    for i, (image_path, (image_rgb, keypoints)) in enumerate(zip(uploaded_image_paths, results_list)):
        print(f"\n🖼️  画像 {i+1}: {image_path}")
        print("-" * 30)
        
        # 各キーポイントの詳細情報
        print("📍 全キーポイントの詳細:")
        for name, idx in KEYPOINT_DICT.items():
            y, x, conf = keypoints[idx]
            status = "✅ 検出" if conf > 0.3 else "❌ 未検出"
            print(f"   {name:15}: {status} (信頼度: {conf:.3f}, 座標: ({x:.3f}, {y:.3f}))")
        
        # 検出率の計算
        detected_count = sum(1 for _, _, conf in keypoints if conf > 0.3)
        detection_rate = detected_count / 17 * 100
        print(f"\n📊 検出率: {detection_rate:.1f}% ({detected_count}/17 キーポイント)")
        
        # 平均信頼度
        avg_confidence = np.mean([conf for _, _, conf in keypoints if conf > 0.3])
        print(f"📈 平均信頼度: {avg_confidence:.3f}")
        
        print()
    
    print("✅ 詳細分析が完了しました！")
else:
    print("❌ 分析する結果がありません")

## 🎯 使用方法のまとめ

1. **セルを順番に実行**: 上から順番にセルを実行してください
2. **画像をアップロード**: セル6で人物が写った画像をアップロードしてください
3. **結果を確認**: セル8で姿勢推定結果を可視化して確認できます
4. **詳細分析**: セル9で各キーポイントの詳細情報を確認できます

## 📝 注意事項

- **最適な画像**: 人物が明確に写っている画像を使用することをお勧めします
- **信頼度閾値**: デフォルトは0.3ですが、必要に応じて調整できます
- **モデルの制限**: 単一人物の姿勢推定にのみ対応しています
- **GPU利用**: Google ColabでGPUランタイムを使用するとより高速に動作します

## 🔗 参考リンク

- [MoveNet: Ultra fast and accurate pose detection model](https://blog.tensorflow.org/2021/05/next-generation-pose-detection-with-movenet-and-tensorflowjs.html)
- [TensorFlow Hub - MoveNet](https://tfhub.dev/google/movenet/singlepose/lightning/4)
- [TensorFlow Pose Estimation Guide](https://www.tensorflow.org/lite/examples/pose_estimation/overview)